## 감정분석
- 감정 또는 감성(sentiment)
- 텍스트에 나타난 긍정/부정의 태도
- 기계학습 방식과 사전 방식

### 감정사전
- 긍정 표현, 부정 표현의 사전
- 해당 분야 전문가가 있으면 데이터 없이도 만들 수 있음
- 기계학습으로도 개발 가능

## 네이버 영화평

- 네이버 영화에서 분석하고 싶은 영화 페이지로 들어감
- 네티즌 별점 클릭
- 페이지 번호 우클릭 후 주소 복사

In [35]:
url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=173123&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'


리뷰 수집

In [12]:
import requests
import lxml.html

# 리뷰와 별점을 모을 빈 리스트를 만든다
reviews = []
scores = []

for page in range(1, 30):  # 1~29페이지까지 반복
    res = requests.get(url.format(page))  # 각 페이지에 접속한다
    root = lxml.html.fromstring(res.text)  # html을 처리한다

    # 리뷰를 가져와 reviews에 추가한다
    for review in root.cssselect('.score_reple p'):
        reviews.append(review.text_content())

    # 별점을 가져와 scores에 추가
    for score in root.cssselect('.score_result .star_score em'):
        scores.append(int(score.text_content()))

표 만들기

In [13]:
import pandas as pd

df = pd.DataFrame({
    'score': scores, 
    'review': reviews
})

In [14]:
df.head()

,score,review
0,10,토니 스타크가 피터 파커를 얼마나 사랑하는지 알 수 있는 영화
1,8,이 글을 보신 모든분들 인생 대박나고 가족 모두가 만수무강 입니다
2,10,유투버들 다 틀렸어 ㅋㅋㅋ멀티버스는 개뿔... 관광버스만 무지허게 나옴 ㅋㅋㅋ 어쨌...
3,10,영화관에서 떠들지 맙시다. 소곤소곤대지 맙시다. 애들 데리고 와서 시끄럽게 하지 맙...
4,9,젠장 캡틴은 어떻게 저걾 맞추는거야????


긍/부정 표시

In [15]:
import numpy as np

In [16]:
df['sentiment'] = np.where(df['score'] > 5, 1, 0)
df.head()

# np.where 파생변수 만들때 사용하는 함수

,score,review,sentiment
0,10,토니 스타크가 피터 파커를 얼마나 사랑하는지 알 수 있는 영화,1
1,8,이 글을 보신 모든분들 인생 대박나고 가족 모두가 만수무강 입니다,1
2,10,유투버들 다 틀렸어 ㅋㅋㅋ멀티버스는 개뿔... 관광버스만 무지허게 나옴 ㅋㅋㅋ 어쨌...,1
3,10,영화관에서 떠들지 맙시다. 소곤소곤대지 맙시다. 애들 데리고 와서 시끄럽게 하지 맙...,1
4,9,젠장 캡틴은 어떻게 저걾 맞추는거야????,1


In [17]:
df.shape

(290, 3)

In [18]:
# 저장
df.to_csv('movie_review.csv', encoding='utf8', index=False)

In [19]:
# 불러오기
df = pd.read_csv('movie_review.csv', encoding='utf8')

한글, 알파벳, 숫자 제외한 문자 제거

In [20]:
import re

In [21]:
!pip install subword_nmt

In [22]:
def remove_non_word(text):
    """한글, 알파벳, 숫자를 제외한 문자를 제거"""
    return re.sub(r'[^가-힣A-z0-9]+', ' ', text)

In [23]:
remove_non_word('Wow, 정말 1도 재미 없다!')

'Wow 정말 1도 재미 없다 '

WPM(Word Piece Model) 학습
 - 구글의 WPM에는 BPE(Byte Pair Encoding) 알고리즘
    - 자연어 처리를 위한 주요 전처리 방법으로 사용

In [24]:
from subword_nmt.learn_bpe import learn_bpe
import io

with open('영화평BPE.txt', 'w', encoding='utf8') as outfile:
    infile = io.StringIO(remove_non_word(' '.join(df['review'])))
    learn_bpe(infile, outfile, 1000)

no pair has frequency >= 2. Stopping


In [25]:
from subword_nmt.apply_bpe import BPE

with open('영화평BPE.txt', encoding='utf8') as f:
    bpe = BPE(f, separator='~')

TDM 만들기

- BoW(Bag of Words)란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법
<img src="TDM.jpg">

In [26]:
def tokenizer_wpm(text):
    text = remove_non_word(text)
    tokens = bpe.process_line(text)
    tokens = tokens.split()
    return [t for t in tokens
            if (not t.endswith('~') and len(t) > 1) or len(t) > 2]

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
cv_wpm = CountVectorizer(max_features=1000, tokenizer=tokenizer_wpm)

In [29]:
tdm = cv_wpm.fit_transform(df['review'])

토큰 빈도

In [30]:
freq = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'n': tdm.sum(axis=0).flat
})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [31]:
freq.sort_values('n').tail(10)

,word,n
25,맞추는거야,29
23,만수무강,29
22,마블,29
29,모든분들,29
38,사랑하는지,29
83,하지,58
74,쿠키영상,58
82,피터,58
85,황석희,58
24,맙시다,145


데이터 분할

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    tdm, df['sentiment'], test_size=.2, random_state=1234)

학습

In [33]:
from sklearn.linear_model import LogisticRegressionCV
model = LogisticRegressionCV(random_state=1234)
model.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

성능 평가

In [34]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test)
accuracy_score(y_test, y_pred)

NotFittedError: This LogisticRegressionCV instance is not fitted yet

계수 분석

In [ ]:
word_coef = pd.DataFrame({
    'word': cv_wpm.get_feature_names(),
    'coef': model.coef_.flat
})

In [ ]:
word_coef.head()

In [ ]:
word_coef.sort_values('coef', ascending=False).head(10)

In [ ]:
word_coef.sort_values('coef').tail(10)